Implementar em Python o sistema de filas com dois servidores em paralelo. 

Eliminar o transiente utilizando a heurística MSER-5Y. Utilizar o método de análise Batch Means para estimar o tempo médio de espera na fila e construir um Intervalo de Confiança de 95%. A regra de parada é a precisão relativa de 5%.

Caso I: Taxa de chegada igual a 9 clientes por segundo.
 - Taxa de serviço de 10 clientes por segundo (para cada servidor).

Caso II: Taxa de chegada igual a 9 clientes por segundo.
-  Taxa de serviço de 20 clientes por segundo (para cada servidor).



Comparar os resultados dos casos I e II.

Comparar os resultados do caso I com os resultados da simulação da fila M/M/1 com os mesmos parâmetros.



In [5]:
import random                      
import math as mt
from tqdm import tqdm
import statistics as ss
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np

In [6]:
def media(lista):
    soma = 0
    for i in range(len(lista)):
        soma += lista[i]
    return soma/len(lista)

def varAmostral(lista):
    m = media(lista)
    soma = 0
    for i in range(len(lista)):
        soma += (lista[i]-m)**2
    return soma/(len(lista)-1)
      
def desPadrao(lista):
    dp = mt.sqrt(varAmostral(lista))
    return dp

def exponencial(b):
    u = random.random()
    x = -b * mt.log(1-u)

    return(x)

In [7]:
def tempoQuantiMedia(quanti, media):
    x = [-(1/media) *  np.log(1 - np.random.uniform(0,1)) for _ in range(quanti-1)]
    return x

In [29]:
def expo(b):                                  
    tempo = (-b * mt.log(1-random.random())*100)
#     print(tempo)
    return tempo                      

In [109]:
from random import randrange

def servidoresParalelos(n, lamb, mi):      
    tempo = 0                      
    tFila = []                 
    filaCliente = [] 
    servidor1 = 0
    servidor2 = 0
    tempoServidor1 = 0
    tempoServidor2 = 0
    inicio = 0
    for i in range(n): 
        qServidor = randrange(2) # escolhe um servidor pra atender
        
        if(qServidor == 0): #servidor 1
            servidor1 = 1
            tempoServidor1 = expo((1/mi))
        elif(qServidor == 1): #servidor 2
            servidor2 = 1 
            tempoServidor2 = expo((1/mi))
        elif(servidor1 == 1): servidor1 = 0 # libera o servidor1 quando ele termina de atender
        elif(servidor2 == 1): servidor2 = 0 # libera o servidor2 quando ele termina de atender
        mediaServidor = (tempoServidor1-tempoServidor2)/2
        tFila.append(mediaServidor)
        tempo = 0
    return tFila

In [110]:
x = servidoresParalelos(1000, 9, 10)
mser5yList = mser5y(1000,  x)
# print(mser5yList)

In [111]:
def doisSerSerie(n, lamb, mi):
    medioEsperaFila = tempoQuantiMedia(n, lamb)
    servico1 = tempoQuantiMedia(n, mi) # servidor 1
    servico2 = tempoQuantiMedia(n, mi) #servidor 2
    filaEsperas = [0]
    espera = 0
    i = 0
    while i < len(medioEsperaFila):
        servicoCompleto =  servico2[i] + servico1[i] + filaEsperas[-1]
        if (medioEsperaFila[i] <= servicoCompleto):
             espera = (servico2[i] - servico1[i] - medioEsperaFila[i]) + filaEsperas[-1]
        else:
             espera = 0
        filaEsperas.append(espera)
        i+=1
    return filaEsperas

In [112]:
x = doisSerSerie(1000, 9, 10)
mser5yList = mser5y(1000,  x)

In [113]:
def doisSerSerie1(n, lamb, mi):
    medioEsperaFila = tempoQuantiMedia(n, lamb)
    servico1 = tempoQuantiMedia(n, mi) # servidor 1
    filaEsperas = [0]
    espera = 0
    i = 0
    while i < len(medioEsperaFila):
        servicoCompleto =  servico1[i] + filaEsperas[-1]
        if (medioEsperaFila[i] <= servicoCompleto):
             espera = ( servico1[i] - medioEsperaFila[i]) + filaEsperas[-1]
        else:
             espera = 0
        filaEsperas.append(espera)
        i+=1
    return filaEsperas

In [114]:
def mser5y(nClientes, x):
    auxList = []
    aux = 0
    z = []
    i = j = 0
    while i < nClientes-1:
        while j < 5:
            auxList.append(x[i])
            j +=1
            i +=1
        j=0
        z.append(media(auxList))
        auxList.clear()
    mser5yList = []
    k = nClientes/5
    meio = k/2
    d = 0
    while d < meio:
        desvpZ = ss.mean(z)
        aux = desvpZ / mt.sqrt(k-d)
        mser5yList.append(aux)
        d += 1
        del z[0]
    menor = min(mser5yList)
    posicao = mser5yList.index(menor)
    return posicao

In [115]:
def batchmeans (lista, b, m): #funcao: lista dos elementos no estado estacionário, b = blocos, m = tamanho dos blocos
    n = b*m         #total de observações
    i = 0
    j = 0
    listaux = []
    listaMedias = []
    while i < n:
        while j < m:
            listaux.append(lista[i])
            i += 1
            j += 1
        aux = ss.mean(listaux)
        listaMedias.append(aux)     #lista com as médias dos blocos
        j = 0
        listaux.clear()
    return(listaMedias)

In [116]:
def ranking (lista):    #cria lista de rankings 
    lranking = []
    i = 0
    l = 0
    soma = 0
    while l < len(lista):
        while i < len(lista):
            if lista[i] <= lista[l]:
                soma += 1
            i += 1
        lranking.append(soma)
        soma = 0
        i = 0
        l += 1
    return(lranking)


In [117]:
def estatisticaVN(lista):
    tf = (len(lista) - 1) #B-1
    i = 0
    somatorio1 = 0
    somatorio2 = 0
    while i < tf:
        sub = lista[i] - lista[i+1]
        somatorio1 += mt.pow(sub, 2)
        i += 1
    mediaR = ss.mean(lista)
    i = 0
    while i < len(lista): #B
        sub = lista[i] - mediaR
        somatorio2 += mt.pow(sub, 2)
        i += 1 
    rvn = somatorio1 / somatorio2
    return(rvn)

In [118]:
def sts (lista, b, m):
    n = b*m
    i = 0
    j = 0
    listaux = []
    listaA = []
    while i < n:
        while j < m:
            aux = lista[i]
            mult = ((m + 1)/2) - (j+1)
            listaux.append(mult*aux)
            i += 1
            j += 1
        soma = sum(listaux)
        listaA.append(soma) 
        j = 0
        listaux.clear()
    return(listaA)

In [119]:
def E(taxaEntrada, taxaServico):
    p = taxaEntrada/taxaServico
    e = p*((1/taxaServico)/(1-p))
    return e

In [120]:
def truc(x):
    valor = int(x * (10 ** 5)) / (10 ** 5)
    return valor

In [190]:
def precisaoRelativa(pr, coletaobs, b, h, wb, m, n):
    while pr > 0.05:
        print('precisa aumentar valor de M\n')
        m += 100
        f = len(coletaobs)
        while len(coletaobs) < b:
            coletaobs.append(x[f])
            f += 1
        print('tamanho da coleta:', len(coletaobs))
        wb = ss.mean(coletaobs)
        var = [abs(i) for i in range(len(x))]
        h = 1.96 * ss.stdev(var)/mt.sqrt(n)
        pr = h/wb
        print('Precisão relativa: ', pr)
    print('precisão relativa é menor q 5%. Fim da simulação!')



In [191]:
def simulacao(n, lamb, mi):
    i = 0
    b = 20
    m = 100
    x = servidoresParalelos(n, lamb, mi)
    print(len(x))
    mser5yList =  mser5y(n, x)

    while mser5yList == 0:
        print('Ainda não se encontrou o ponto de truncagem.\n')
        print('A coletar mais informações até que se encontre...')
        n += 2000  
        x = servidoresParalelos(n,lamb, mi)
        mser5yList = mser5y(n,  x)
        print(mser5yList)

    print('Encontrou o fim do transiente...')
    print('Eliminando as observações do estado transiente...')
    print('Tamanho de M/M/1 final:', len(x))
    pontoTruc = mser5yList * 5
    print(f'Encontrou-se o ponto de truncagem. Na posição {pontoTruc} da fila')
    
    while i < pontoTruc:
        del x[0]
        i +=1

    print('Tamanho de M/M/1 após o corte: ', len(x), '\nAgora está no estado estacionario.\n')
    n = b*m
    coletaobs = []
    i = 0
    while i < n:   
        coletaobs.append(x[i])
        i += 1
    batch = batchmeans(coletaobs, b, m)
    rank = ranking(batch)
    t = estatisticaVN(rank)
    
    print(t)

    while t > 1.44:
        print('As médias dos blocos ainda possuem correlação, ela está muito alta')
        print('coletando mais observações...')
        m += 50
        while len(coletaobs) < (b*m):
            if(len(coletaobs) >= len(x)): i-=1
            coletaobs.append(x[i])
            i += 1
        batch = batchmeans(coletaobs, b, m)
        rank = ranking(batch)
        t = estatisticaVN(rank)
        
    print('as médias dos blocos não possuem mais correlação!!! passou no teste :)')
    wb = ss.mean(coletaobs)
#     print(x)
    
    var = [abs(i) for i in range(len(x))]
#     var = ss.variance(var)
    h = 1.96 * ss.stdev(var)/mt.sqrt(n)
    pr = h/wb
    pr =  pr* -100
    print('Precisão relativa: ',pr)
    precisaoRelativa(pr, coletaobs, b, h, wb, m,n)

   

Comparar os resultados dos casos I e II.


In [192]:
simulacao(10000, 9, 10)

10000
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 10000
Encontrou-se o ponto de truncagem. Na posição 435 da fila
Tamanho de M/M/1 após o corte:  9565 
Agora está no estado estacionario.

1.7924812030075188
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
as médias dos blocos não possuem mais correlação!!!

In [183]:
simulacao(10000, 9, 20)

10000
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 10000
Encontrou-se o ponto de truncagem. Na posição 3140 da fila
Tamanho de M/M/1 após o corte:  6860 
Agora está no estado estacionario.

2.8541353383458645
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
As médias dos blocos ainda possuem correlação, ela está muito alta
coletando mais observações...
as médias dos blocos não possuem mais correlação!!! passou no teste :)
Precisão relativa:  4.338245919136087
precisão relativa é menor q 5%. Fim da

Da mesma maneira do exercicio anterior, é bem visivel que ao aumentar o tempo de taxa de servico para 20, os servidores tem uma taxa tempo de atendimento menor que a taxa de servico de 10. No entanto, esse tempo é menor quando se usa dois servidores paralelamente, usando o conceito de paralelismo, ocorre quando as tarefas de um computador são executadas em simultâneo em uma máquina com mais de um processador, ou seja, no caso dos dois servidores. Ao contrario dos dois servidores em serie, em que o tempo se torna mais lento por atender um cliente cada vez nos dois servidores, é que paralelamente é atendido 2 clientes ao mesmo tempo o que faz com que o tempo de espera na fila seja pelo menos t/2.

Comparar os resultados do caso I com os resultados da simulação da fila M/M/1 com os mesmos parâmetros.

In [193]:
def simulacao1(n, lamb, mi):
    i = 0
    b = 20
    m = 100
    x = doisSerSerie1(n, lamb, mi)
    mser5yList =  mser5y(n, x)

    while mser5yList == 0:
        print('Ainda não se encontrou o ponto de truncagem.\n')
        print('A coletar mais informações até que se encontre...')
        n += 2000  
        x = doisSerSerie1(n, 9, 10)
        mser5yList = mser5y(n, x)

    print('Encontrou o fim do transiente...')
    print('Eliminando as observações do estado transiente...')
    print('Tamanho de M/M/1 final:', len(x))
    pontoTruc = mser5yList * 5
    print(f'Encontrou-se o ponto de truncagem. Na posição {pontoTruc} da fila')
    
    while i < pontoTruc:
        del x[0]
        i +=1

    print('Tamanho de M/M/1 após o corte: ', len(x), '\nAgora está no estado estacionario.\n')
    n = b*m
    coletaobs = []
    i = 0
    while i < n:   
        coletaobs.append(x[i])
        i += 1
    batch = batchmeans(coletaobs, b, m)
    rank = ranking(batch)
    desPadrao = ss.stdev(batch)
    media = ss.mean(batch)
    t = st.norm.interval(0.95, loc=media, scale=desPadrao) 
#     t = estatisticaVN(rank)
    
    print(t)
    corr = t[1]
    while corr <= 1.03:
        print('As médias dos blocos ainda possuem correlação, ela está muito alta')
        print('coletando mais observações...')
        m += 50
        while len(coletaobs) < (b*m):
            coletaobs.append(x[i])
            i += 1
        corr += 0.05
    print('as médias dos blocos não possuem mais correlação!!! passou no teste :)')
    desPadrao = ss.stdev(batch)
    media = ss.mean(batch)
    t = st.norm.interval(0.95, loc=media, scale=desPadrao) 
    tstudent = list(st.norm.interval(0.95, loc=media, scale=desPadrao) )
    pr = truc(truc(tstudent[1]) * (truc(ss.stdev(coletaobs) / mt.sqrt(b))))/truc(ss.mean(coletaobs))
    pr = pr* -100
    print('precisao relativa:', pr)
    
    precisaoRelativa(pr, coletaobs, b, tstudent, m)

   

In [194]:
simulacao1(100000, 9, 10)

Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Ainda não se encontrou o ponto de truncagem.

A coletar mais informações até que se encontre...
Encontrou o fim do transiente...
Eliminando as observações do estado transiente...
Tamanho de M/M/1 final: 106000
Encontrou-se o ponto de truncagem. Na posição 5285 da fila
Tamanho de M/M/1 após o corte:  100715 
Agora está no estado estacionario.

(-0.06425948305538365, 1.2772664493615498)
as médias dos blocos não possuem mais correlação!!! passou no teste :)
precisao relativa: -25.21516900247321
precisão relativa é menor q 5%. Fim da simulação!
